In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [19]:
device = torch.device('cuda')
transform = transforms.Compose([
    transforms.ToTensor(),
    #transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

In [20]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

In [21]:
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())    #adam优化器

In [22]:
train_losses = []
train_accuracies = []

In [23]:
for epoch in range(5):  # 只训练5个epoch
    model.train()
    running_loss=0.0
    correct=0
    total=0
    
    for data, target in train_loader:
        data, target=data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output=model(data)
        loss=criterion(output, target)
        loss.backward()
        optimizer.step()
        
        running_loss+=loss.item()
        _, predicted=torch.max(output.data, 1)
        total+=target.size(0)
        correct+=(predicted == target).sum().item()
    
    epoch_loss=running_loss / len(train_loader)
    epoch_accuracy=100 * correct / total
    train_losses.append(epoch_loss)
    train_accuracies.append(epoch_accuracy)
    
    print(f'{epoch+1}: loss: {epoch_loss:.4f}, ACC: {epoch_accuracy:.2f}%')

Epoch 1/5: loss: 0.3735, ACC: 88.27%
Epoch 2/5: loss: 0.1178, ACC: 96.53%
Epoch 3/5: loss: 0.0878, ACC: 97.46%
Epoch 4/5: loss: 0.0710, ACC: 97.95%
Epoch 5/5: loss: 0.0599, ACC: 98.22%


In [18]:
model.eval()
test_correct = 0
test_total = 0
all_predictions = []
all_targets = []

with torch.no_grad():
    for data, target in test_loader:
        data, target=data.to(device), target.to(device)
        output=model(data)
        _, predicted=torch.max(output.data, 1)
        test_total+=target.size(0)
        test_correct+=(predicted==target).sum().item()
        all_predictions.extend(predicted.cpu().numpy())
        all_targets.extend(target.cpu().numpy())

test_accuracy=100 * test_correct / test_total
print(f'\nACC: {test_accuracy:.2f}%')


ACC: 98.98%
